Test

In [1]:
# Step 1: Launch a Colab Notebook and Set Up Environment
# Install necessary packages first:
%pip install torch torchvision torchaudio transformers accelerate bitsandbytes huggingface_hub scikit-learn pandas numpy matplotlib --quiet

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install transformers datasets bitsandbytes accelerate huggingface_hub --quiet

Note: you may need to restart the kernel to use updated packages.


In [1]:
# Step 2: Load and Run Mistral Model from Hugging Face
# You can directly load Mistral-7B-Instruct-v0.1 from Hugging Face.
# This model supports instruction-based prompting similar to GPT-based models.

import os
from huggingface_hub import login

# Hugging Face access token: x

os.environ["HUGGINGFACE_TOKEN"] = "x"
login(token=os.environ["HUGGINGFACE_TOKEN"])

from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.1"

quant_config = BitsAndBytesConfig(load_in_4bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, token=os.environ["HUGGINGFACE_TOKEN"])
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quant_config,
    device_map="auto",
    token=os.environ["HUGGINGFACE_TOKEN"]
)

text_gen_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


HTTPError: Invalid user token.

In [ ]:
def build_prompt(start_id, num_rows):
    return f"""
Generate ONLY CSV data without any explanation or markdown.
The CSV must contain exactly {num_rows} rows (excluding header) with the following columns:
customer_id, age, income, transaction_amount, industry, sentiment, risk_category, customer_segment, decision

Ensure that:
- customer_id starts from {start_id} and increments by 1
- age values are between 18 and 65
- income ranges from 30000 to 150000
- transaction_amount ranges from 50 to 20000
- industry is a realistic business sector (Retail, Finance, Tech, Healthcare, Manufacturing)
- sentiment is one of: positive, neutral, negative
- risk_category is one of: low, medium, high and logically correlates with income and transaction patterns
- customer_segment is one of: budget, standard, premium, enterprise
- decision is yes/no with a roughly equal distribution
- logical consistency exists across fields

Output CSV rows ONLY.
"""


Batch Generation + Save CSV

In [3]:
import csv

BATCH_SIZE = 200
TOTAL_ROWS = 1000
NUM_BATCHES = TOTAL_ROWS // BATCH_SIZE

header = [
    "customer_id", "age", "income", "transaction_amount",
    "industry", "sentiment", "risk_category",
    "customer_segment", "decision"
]

all_rows = []

for batch in range(NUM_BATCHES):
    start_id = batch * BATCH_SIZE + 1
    prompt = build_prompt(start_id, BATCH_SIZE)

    print(f"Generating batch {batch + 1}/{NUM_BATCHES}...")

    response = text_gen_pipeline(
        prompt,
        max_new_tokens=1200,   # important for CSV completion
        do_sample=True,
        temperature=0.7,
        top_p=0.9
    )

    generated_text = response[0]["generated_text"].strip()

    rows = generated_text.split("\n")

    for row in rows:
        values = row.split(",")
        if len(values) == len(header):
            all_rows.append(values)

print(f"✅ Total valid rows collected: {len(all_rows)}")


NameError: name 'build_prompt' is not defined

Save file to csv

In [ ]:
output_file = "synthetic_financial_data.csv"

with open(output_file, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(all_rows)

print(f"📁 CSV saved successfully: {output_file}")
# Step 3: Save Generated Data to CSV